In [1]:
from herbie import Herbie
import herbie
#from toolbox import EasyMap, pc
#from paint.standard2 import cm_tmp
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import shapely.geometry as sgeom
import cartopy.feature
import cartopy.feature as cfeature
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
import numpy as np
#from toolbox.cartopy_tools import common_features, pc
import pandas as pd
import numpy as np
import glob
import matplotlib.cm as cm
import xarray as xr
from datetime import datetime, timedelta
import time
import os
import warnings
warnings.filterwarnings("ignore", message="Will not remove GRIB file because it previously existed.")




"""

HRRR model documentation:
https://rapidrefresh.noaa.gov/hrrr/

Documentation for the HRRR data download wrapper:
https://herbie.readthedocs.io/en/stable/

Variables:
https://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfnatf00.grib2.shtml


"""

'\n\nHRRR model documentation:\nhttps://rapidrefresh.noaa.gov/hrrr/\n\nDocumentation for the HRRR data download wrapper:\nhttps://herbie.readthedocs.io/en/stable/\n\nVariables:\nhttps://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfnatf00.grib2.shtml\n\n\n'

In [2]:
#%% Define

# Folder to save datasets
save_folder = "/kfs2/projects/sfcwinds/HRRR"
# save_folder = "../data/HRRR"

# Define download area

# # Have not found a way to slice by lat and lon directly, use x,y instead (below)
# min_lat = 35
# max_lat = 38
# min_lon = -111
# max_lon = -118


# Define area to download (boundaries of download area)
area = "US_SW"   #   "NM_AZ", "US", "US_SW"

if area == "NM_AZ":   # New Mexico and Arizona
    slicex = slice(300, 800)
    slicey = slice(300, 600)

elif area == "US":  # Entire continental US
    slicex = slice(130, 1730)
    slicey = slice(50, 1040)       

elif area == "US_SW":  # US Southwest
    slicex = slice(130, 1000)
    slicey = slice(50, 800)    


# date range with one-day frequency
date_range = pd.date_range(datetime(2014, 1, 1), 
                           datetime(2024, 7, 31), 
                           freq="D").tolist()[::-1]   # list starts from end

# Forecast hour (0=analysis, 2 is recommended)
fxx = 2    

# Chunk dictionaries
chunk_dict_hourly = {"valid_time": 1, "isobaricInhPa": 1, "y": 200, "x": 200}
chunk_dict_daily = {"valid_time": 12, "isobaricInhPa": 1, "y": 200, "x": 200}




# Define compression settings
comp = {"zlib": True, "complevel": 4}  

In [3]:
#%% Functions


def safe_xarray(herbie_obj, search_str):
    
    """ accessing the Herbie object sometimes gives a permission error, adding a wait time helps."""
    
    for _ in range(5):  # Retry up to 5 times
        try:
            return herbie_obj.xarray(search_str, remove_grib=True)
        except PermissionError:
            time.sleep(1)  # Wait and retry
    raise Exception(f"Failed to load {search_str} after multiple attempts.")
    
    
def process_hourly_data(timestamp, area, slicey, slicex, fxx):
    """Retrieve, process, and return hourly HRRR dataset"""
    
    max_retries = 10  # Set a limit to avoid infinite loops
    attempt = 0
    
    while attempt < max_retries:
        
        try:
            valid_time = timestamp + pd.to_timedelta(fxx, "h") # 2-hour shift (forecast lead time fxx)
    
            print(f"{valid_time}")
    
            H_sfc = Herbie(timestamp, model='hrrr', product='sfc', fxx=fxx, verbose=False)
            H_subh = Herbie(timestamp, model='hrrr', product='subh', fxx=fxx, verbose=False)
            
            #H_sfc.inventory().variable.values   
            #H_sfc.inventory()[["variable", "search_this"]].values 
    
            # Define search patterns for variables
            # df = H_sfc.inventory()[["variable", "search_this"]]
            # filtered_df = df[df["search_this"].str.contains(r"\b(?:ugrd|vgrd)\b", case=False, na=False) & 
            #                  df["search_this"].str.contains("mb", case=False, na=False)]
            # wind_search_string_sfc = "|".join(filtered_df["search_this"].drop_duplicates().values)
            wind_search_string_sfc = ':UGRD:250 mb:2 hour fcst|:VGRD:250 mb:2 hour fcst|:UGRD:500 mb:2 hour fcst|:VGRD:500 mb:2 hour fcst|:UGRD:850 mb:2 hour fcst|:VGRD:850 mb:2 hour fcst'

            
            
            df = H_subh.inventory()[["variable", "search_this"]]
            filtered_df = df[df["search_this"].str.contains(r"\b(?:ugrd|vgrd)\b", case=False, na=False) & 
                             df["search_this"].str.contains("ave", case=False, na=False)]
            wind_search_string_subh = "|".join(filtered_df["search_this"].drop_duplicates().values)
                    
    
            
            # Surface data
            ds_sfc = xr.merge([
                safe_xarray(H_sfc, wind_search_string_sfc),
                safe_xarray(H_sfc, ':TMP:2 m'),
                safe_xarray(H_sfc, 'GUST|:PRES:surface:2 hour fcst|:SNOWC:surface:2 hour fcst|:APCP:surface:0-2 hour acc fcst|:SFCR:surface:2 hour fcst|:VEG:surface:2 hour fcst|:VGTYP:surface:2 hour fcst|:HPBL:surface:2 hour fcst'),
                safe_xarray(H_sfc, '[U|V]GRD:10 m'),
                safe_xarray(H_sfc, '[U|V]GRD:80 m').rename({"u": "u80", "v": "v80"}),
                safe_xarray(H_sfc, ':DPT:2 m above ground:2 hour fcst'),
                safe_xarray(H_sfc, ':TCDC:boundary layer cloud layer:2 hour fcst'),
                safe_xarray(H_sfc, ':CAPE:0-3000 m above ground:2 hour fcst')
            ], compat='minimal')
    
            ds_sfc = ds_sfc.sel(y=slicey, x=slicex).expand_dims(dim="valid_time")
            
            # ds_sfc['wspd10'] = (ds_sfc.u10**2 + ds_sfc.v10**2)**0.5
            # ds_sfc['wdir10'] = np.degrees(np.arctan2(ds_sfc.u10, ds_sfc.v10)) +180
            
            ds_sfc = ds_sfc.rename({"u10": "u10_h",
                                    "v10": "v10_h",
                                    # "wspd10": "wspd10_h",
                                    # "wdir10": "wdir10_h"                              
                                    })
    
            # Subhourly data
            ds_subh = xr.merge([
                safe_xarray(H_subh, wind_search_string_subh),
                # safe_xarray(H_subh, ':TMP:2 m|DPT')#.isel(step=[0,1,2])
                safe_xarray(H_subh, 'GUST')
            ], compat='minimal')
    
            ds_subh = ds_subh.sel(y=slicey, x=slicex).swap_dims({"step": "valid_time"})
            
            
            # # get native data during this day - probably not needed for surface wind analysis
            # H_nat = herbie.fast.FastHerbie(
            #         time_range[:],
            #         prioriy = 'google',
            #         model="hrrr",
            #         product="nat", # to get 15min steps backwards, set fxx to 1 and product "subh", otherwise "sfc", alternatively "nat"
            #         fxx=[2]
            #     )
           
            # #H_nat.inventory()[["variable", "search_this"]].values 
                  
            # # download seperate xarrays for different search strings 
            # ds1 = safe_xarray(H_nat, ':UGRD:1 hybrid level:2 hour fcst|:VGRD:1 hybrid level:2 hour fcst')
            # ds1 = ds1.sel(y=slicey, x=slicex)  
           
            # ds_nat = xr.merge([ds1],compat='minimal')
       
            # # Add the time dimension
            # ds_nat = ds_nat.expand_dims(dim="valid_time")
            
    
            # Merge hourly and subhourly datasets
            hrrr = xr.merge([ds_sfc, ds_subh], compat='override')
    
            # Cleanup variables and attributes
            hrrr = hrrr.drop_vars(["boundaryLayerCloudLayer", "heightAboveGroundLayer", 
                                   "surface", "gribfile_projection", "time"], errors="ignore")
            hrrr.attrs = {}
            for var in hrrr.data_vars:
                hrrr[var].attrs = {}
    
            return hrrr.chunk(chunk_dict_hourly)  # Apply chunking to hourly files
    
        except Exception as e:
    
            print(f"Error: {e}")
            
            attempt += 1
            print(f"Attempt {attempt} failed. Retrying...")
            time.sleep(1)  # Wait before retrying        




In [ ]:
if __name__ == "__main__":
    
    t1 = datetime.now()
    
    # loop over days in date_range
    for date in date_range[:]: 
        
        print(date)
        
        daily_file_path = os.path.join(save_folder, f"hrrr_{area}_{date.date()}.nc")
        
        # Check if file already exists, exit loop if so
        if os.path.exists(daily_file_path):
            print(f"File already exists: {daily_file_path}.")
            continue
        
        daily_datasets = []  # Store hourly datasets for the day
        try:
            del daily_hrrr
        except:
            pass
    
        for hour in range(-fxx+1, fxx+21, 1):  # Looping to hours, so that valid_date goes from 0h to 24h
            
            timestamp = date + pd.to_timedelta(hour, "h")
                 
            # Load and process  hourly files      
            hrrr = process_hourly_data(timestamp, area, slicey, slicex, fxx)
            
            if hrrr is not None:

                # Store dataset in list
                daily_datasets.append(hrrr)   
                          
                # # Save hourly file
                # valid_time = timestamp + pd.to_timedelta(fxx, "h")
                # hourly_file_path = os.path.join(save_folder, f"hrrr_{area}_{valid_time.date()}_{valid_time.hour:02d}h.nc")
                # encoding = {
                #     var: {**comp, "chunksizes": (1, 1, 200, 200)} # 
                #     if len(hrrr[var].dims) == 4 else {**comp, "chunksizes": (1, 200, 200)}
                #     for var in hrrr.data_vars
                #     }
                # hrrr.to_netcdf(hourly_file_path, encoding=encoding) # '../data/HRRR\\hrrr_test.nc'
                # # load: hrrr = xr.open_dataset(save_folder+"/hrrr_NM_AZ_2024-12-31_00h.nc", chunks="auto")
                # print ("Saved hourly file.")

        # print ("hourly files done")
        
    
        # Merge all hourly datasets into a daily file
        daily_hrrr = xr.concat(daily_datasets, dim="valid_time", combine_attrs="override") 
        daily_hrrr = daily_hrrr.chunk(chunk_dict_daily)
        
            
        # # save daily file
        encoding = {
            var: {**comp, "chunksizes": (12, 1, 200, 200)} # 
            if len(hrrr[var].dims) == 4 else {**comp, "chunksizes": (1, 200, 200)}
            for var in hrrr.data_vars
            }
        daily_hrrr.to_netcdf(daily_file_path, encoding=encoding)
        print ("daily file saved")
        
    t2 = datetime.now()
    
    print (t2-t1)

2024-07-31 00:00:00
File already exists: /kfs2/projects/sfcwinds/HRRR/hrrr_US_SW_2024-07-31.nc.
2024-07-30 00:00:00
File already exists: /kfs2/projects/sfcwinds/HRRR/hrrr_US_SW_2024-07-30.nc.
2024-07-29 00:00:00
File already exists: /kfs2/projects/sfcwinds/HRRR/hrrr_US_SW_2024-07-29.nc.
2024-07-28 00:00:00
File already exists: /kfs2/projects/sfcwinds/HRRR/hrrr_US_SW_2024-07-28.nc.
2024-07-27 00:00:00
File already exists: /kfs2/projects/sfcwinds/HRRR/hrrr_US_SW_2024-07-27.nc.
2024-07-26 00:00:00
File already exists: /kfs2/projects/sfcwinds/HRRR/hrrr_US_SW_2024-07-26.nc.
2024-07-25 00:00:00
File already exists: /kfs2/projects/sfcwinds/HRRR/hrrr_US_SW_2024-07-25.nc.
2024-07-24 00:00:00
File already exists: /kfs2/projects/sfcwinds/HRRR/hrrr_US_SW_2024-07-24.nc.
2024-07-23 00:00:00
File already exists: /kfs2/projects/sfcwinds/HRRR/hrrr_US_SW_2024-07-23.nc.
2024-07-22 00:00:00
File already exists: /kfs2/projects/sfcwinds/HRRR/hrrr_US_SW_2024-07-22.nc.
2024-07-21 00:00:00
File already exists:

In [31]:
# test hourly vs subourly wind speed
plt.figure()
plt.plot(daily_hrrr.valid_time.values, daily_hrrr.isel(x=100, y=100).u10_h.values, ".", color="blue", label = "HRRR sfc")    
plt.plot(daily_hrrr.valid_time.values, daily_hrrr.isel(x=100, y=100).u10.values, ".", color="orange", label = "HRRR subh")    

NameError: name 'daily_hrrr' is not defined

<Figure size 640x480 with 0 Axes>

In [6]:
# old without functions    

if __name__ == "__main__":

    # read HRRR in time range

    # loop over days in date_range
    for date in date_range[:1]: 
    
        print(date)
        
        daily_file_path = os.path.join(save_folder, f"hrrr_{area}_{date.date()}.nc")
        
        # # Check if file already exists, exit loop if so
        # if os.path.exists(daily_file_path):
        #      print(f"File already exists: {daily_file_path}.")
        #      continue
        
        daily_datasets = []  # Store hourly datasets for the day
        try:
            del daily_hrrr
        except:
            pass
        
        for hour in range(-1, 23, 1):  # Looping to cover 0h-22h, so valid_date goes up to 24h
            
            try:
            
                 timestamp = date + pd.to_timedelta(hour, "h") 
                 valid_time = timestamp + pd.to_timedelta(fxx, "h")  # 2-hour shift (forecast lead time fxx)
        

                
                 print (valid_time)
        
                 # Define file path
                 file_path = os.path.join(save_folder, f"hrrr_{area}_{valid_time.date()}_{valid_time.hour:02d}h.nc")
                
                 print("sfc file")
            
                 # get hourly surface data during this day
                 H_sfc = Herbie(
                      timestamp,
                     # priority = 'google',
                      model='hrrr',
                      product='sfc',
                      fxx=fxx,
                      verbose=False
                    )
                                    
                 #H_sfc.file_exists   
                 #H_sfc.inventory().variable.values   
                 #H_sfc.inventory()[["variable", "search_this"]].values    
                 # H.xarray("(?:HGT|LAND):surface")
                       
                 # download seperate xarrays for different search strings (why cannot search with one search string? - maybe different steps for each variable?)   
                 df = H_sfc.inventory()[["variable", "search_this"]]
                 filtered_df = df[df["search_this"].str.contains(r"\b(?:ugrd|vgrd)\b", case=False, na=False) & 
                                  df["search_this"].str.contains("mb", case=False, na=False)]
                 wind_search_string = "|".join(filtered_df["search_this"].drop_duplicates().values)
                
                 ds1 = safe_xarray(H_sfc, wind_search_string)
                 ds2 = safe_xarray(H_sfc, ':TMP:2 m')
                 ds3 = safe_xarray(H_sfc, 'GUST|:PRES:surface:2 hour fcst|:SNOWC:surface:2 hour fcst|:APCP:surface:0-2 hour acc fcst|:SFCR:surface:2 hour fcst|:VEG:surface:2 hour fcst|:VGTYP:surface:2 hour fcst|:HPBL:surface:2 hour fcst')
                 ds4 = safe_xarray(H_sfc, '[U\|V]GRD:10 m')
                 ds5 = safe_xarray(H_sfc, '[U\|V]GRD:80 m')
                 ds5 = ds5.rename({"u": "u80",
                                         "v": "v80",                              
                                         })
                 ds6 = safe_xarray(H_sfc, ':DPT:2 m above ground:2 hour fcst')
                 ds7 = safe_xarray(H_sfc, ':TCDC:boundary layer cloud layer:2 hour fcst')
                 ds8 = safe_xarray(H_sfc, ':CAPE:0-3000 m above ground:2 hour fcst')
                
        
                 ds1 = ds1.sel(y=slicey, x=slicex)  
                 ds2 = ds2.sel(y=slicey, x=slicex)  
                 ds3 = ds3.sel(y=slicey, x=slicex) 
                 ds4 = ds4.sel(y=slicey, x=slicex)
                 ds5 = ds5.sel(y=slicey, x=slicex)
                 ds6 = ds6.sel(y=slicey, x=slicex) 
                 ds7 = ds7.sel(y=slicey, x=slicex)
                 ds8 = ds8.sel(y=slicey, x=slicex)            
        
                
                 ds_sfc = xr.merge([ds1,ds2, ds3, ds4, ds5, ds6, ds7, ds8],compat='minimal')
                
                 # ds_sfc['wspd10'] = (ds_sfc.u10**2 + ds_sfc.v10**2)**0.5
                 # ds_sfc['wdir10'] = np.degrees(np.arctan2(ds_sfc.u10, ds_sfc.v10)) +180
                
                
                 ds_sfc = ds_sfc.rename({"u10": "u10_h",
                                         "v10": "v10_h",
                                         # "wspd10": "wspd10_h",
                                         # "wdir10": "wdir10_h"                              
                                         })
                
                 # Add the time dimension
                 ds_sfc = ds_sfc.expand_dims(dim="valid_time")
                
                
                 print("subh file")
                
                 # get subhourly data during this day                
                 H_subh = Herbie(
                      timestamp,
                      # priority = 'google',
                      model='hrrr',
                      product='subh',
                      fxx=fxx,
                      verbose=False
                    )
                
                
                 #H_subh.inventory()[["variable", "search_this"]].values 
                
                 # Extract wind variables
                 df = H_subh.inventory()[["variable", "search_this"]]
                 filtered_df = df[df["search_this"].str.contains(r"\b(?:ugrd|vgrd)\b", case=False, na=False) & 
                                  df["search_this"].str.contains("ave", case=False, na=False)]
                 wind_search_string = "|".join(filtered_df["search_this"].drop_duplicates().values)
                
           
                 ds1 = safe_xarray(H_subh, wind_search_string)#.isel(step=[0,1,2]) # skip the last step because this is the forecast for the next hour
                # ds2 = safe_xarray(H_subh, ':TMP:2 m|DPT')#.isel(step=[0,1,2])
                 ds3 = safe_xarray(H_subh, 'GUST')#.isel(step=[0,1,2])
                
                 ds1 = ds1.sel(y=slicey, x=slicex)   
                 #ds2 = ds2.sel(y=slicey, x=slicex)  
                 ds3 = ds3.sel(y=slicey, x=slicex)   
                
                 ds_subh = xr.merge([ds1, ds3],compat='minimal')
                
                 # ds_subh['wspd10'] = (ds_subh.u10**2 + ds_subh.v10**2)**0.5
                 # ds_subh['wdir10'] = np.degrees(np.arctan2(ds_subh.u10, ds_subh.v10)) +180
            
                 ds_subh = ds_subh.swap_dims({"step": "valid_time"})   
                
                    
                 # # get native data during this day - probably not needed for surface wind analysis
                 # H_nat = herbie.fast.FastHerbie(
                 #         time_range[:],
                 #         prioriy = 'google',
                 #         model="hrrr",
                 #         product="nat", # to get 15min steps backwards, set fxx to 1 and product "subh", otherwise "sfc", alternatively "nat"
                 #         fxx=[2]
                 #     )
                
                 # #H_nat.inventory()[["variable", "search_this"]].values 
                       
                 # # download seperate xarrays for different search strings 
                 # ds1 = safe_xarray(H_nat, ':UGRD:1 hybrid level:2 hour fcst|:VGRD:1 hybrid level:2 hour fcst')
                 # ds1 = ds1.sel(y=slicey, x=slicex)  
                
                 # ds_nat = xr.merge([ds1],compat='minimal')
            
                 # # Add the time dimension
                 # ds_nat = ds_nat.expand_dims(dim="valid_time")
            
                 print("merge")
          
                 # Merge hourly surface files and subhourly files
                 hrrr = xr.merge([ds_sfc, ds_subh], compat='override')  # storage space is less when saving in combines datafile
            
                
                 # Reduce file size (drop vars, chunking, compression)
                 hrrr = hrrr.drop_vars(["boundaryLayerCloudLayer", "heightAboveGroundLayer", "surface", "gribfile_projection", "time"], errors="ignore")
                 hrrr.attrs = {}
                 for var in hrrr.data_vars:
                     hrrr[var].attrs = {}
                
                
                 # Chunk hourly files
                 hrrr = hrrr.chunk(chunks="auto")
                    
                 # Store dataset in list
                 daily_datasets.append(hrrr)
                    
                 # hrrr.to_netcdf(file_path, encoding=encoding) # '../data/HRRR\\hrrr_test.nc'
                 # # load: hrrr = xr.open_dataset(save_folder+"/hrrr_NM_AZ_2024-12-31_00h.nc", chunks="auto")
                 # print ("Saved hourly file.")
                        
            except:
                
                attempt = 0
                
                while attempt < max_retries:
                    attempt += 1
                    print(f"Attempt {attempt} failed. Retrying...")
                    time.sleep(1)  # Wait before retrying
        
        print ("hourly files done")
        
        chunk_dict = {"valid_time": 12, "isobaricInhPa": 1, "y": 200, "x": 200}
        
        # Apply consistent chunking to each dataset in daily_datasets
        for i in range(len(daily_datasets)):
            daily_datasets[i] = daily_datasets[i].chunk(chunk_dict)
        
        
        # Merge all hourly datasets into a daily file
        daily_hrrr = xr.concat(daily_datasets, dim="valid_time", combine_attrs="override") 
        daily_hrrr = daily_hrrr.chunk(chunk_dict)
        
            
        # save daily file
        comp = {"zlib": True, "complevel": 9}  # 1 being fastest, but lowest compression ratio, 9 being slowest but best compression ratio  
        # encoding = {
        #             var: {**comp, "chunksizes": (4, 2, 100, 100)} # 
        #             if len(hrrr[var].dims) == 4 else {**comp, "chunksizes": (4, 100, 100)}
        #             for var in hrrr.data_vars
        #         }
        encoding = {var: {**comp, "chunksizes": None} for var in daily_hrrr.data_vars}
        #daily_hrrr.to_netcdf(daily_file_path, encoding=encoding)
        print ("daily file saved")
           
        # # Load the files:
        # file_pattern = os.path.join(save_folder, f"hrrr_{area}_{valid_time.date()}_*h.nc")
        # file_list = sorted(glob.glob(file_pattern))
        # if file_list:
        #     hrrr = xr.open_mfdataset(file_list[:], concat_dim ="valid_time",combine='nested', chunks="auto", parallel = True)
        
        
        # test hourly vs subourly wind speed
        # plt.figure()
        # plt.plot(daily_hrrr.valid_time.values, daily_hrrr.isel(x=100, y=100).u10_h.values, ".", color="blue", label = "HRRR sfc")    
        # plt.plot(daily_hrrr.valid_time.values, daily_hrrr.isel(x=100, y=100).u10.values, ".", color="orange", label = "HRRR subh")  

<>:64: SyntaxWarning: invalid escape sequence '\|'
<>:65: SyntaxWarning: invalid escape sequence '\|'
<>:64: SyntaxWarning: invalid escape sequence '\|'
<>:65: SyntaxWarning: invalid escape sequence '\|'
/tmp/scratch/7410880/ipykernel_555495/1114614423.py:64: SyntaxWarning: invalid escape sequence '\|'
  ds4 = safe_xarray(H_sfc, '[U\|V]GRD:10 m')
/tmp/scratch/7410880/ipykernel_555495/1114614423.py:65: SyntaxWarning: invalid escape sequence '\|'
  ds5 = safe_xarray(H_sfc, '[U\|V]GRD:80 m')
/tmp/scratch/7410880/ipykernel_555495/1114614423.py:27: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  timestamp = date + pd.to_timedelta(hour, "H")
/tmp/scratch/7410880/ipykernel_555495/1114614423.py:28: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  valid_time = timestamp + pd.to_timedelta(fxx, "H")  # 2-hour shift (forecast lead time fxx)


2024-12-31 00:00:00
2024-12-31 01:00:00
sfc file
subh file
merge


/tmp/scratch/7410880/ipykernel_555495/1114614423.py:27: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  timestamp = date + pd.to_timedelta(hour, "H")
/tmp/scratch/7410880/ipykernel_555495/1114614423.py:28: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  valid_time = timestamp + pd.to_timedelta(fxx, "H")  # 2-hour shift (forecast lead time fxx)


2024-12-31 02:00:00
sfc file
subh file
merge


/tmp/scratch/7410880/ipykernel_555495/1114614423.py:27: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  timestamp = date + pd.to_timedelta(hour, "H")
/tmp/scratch/7410880/ipykernel_555495/1114614423.py:28: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  valid_time = timestamp + pd.to_timedelta(fxx, "H")  # 2-hour shift (forecast lead time fxx)


...

2024-12-31 22:00:00
sfc file
subh file
merge


/tmp/scratch/7410880/ipykernel_555495/1114614423.py:27: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  timestamp = date + pd.to_timedelta(hour, "H")
/tmp/scratch/7410880/ipykernel_555495/1114614423.py:28: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  valid_time = timestamp + pd.to_timedelta(fxx, "H")  # 2-hour shift (forecast lead time fxx)


2024-12-31 23:00:00
sfc file
subh file
merge


/tmp/scratch/7410880/ipykernel_555495/1114614423.py:27: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  timestamp = date + pd.to_timedelta(hour, "H")
/tmp/scratch/7410880/ipykernel_555495/1114614423.py:28: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  valid_time = timestamp + pd.to_timedelta(fxx, "H")  # 2-hour shift (forecast lead time fxx)


2025-01-01 00:00:00
sfc file
subh file
merge
hourly files done
daily file saved


In [23]:
# Test different nc save configurations

test = daily_hrrr#.isel(valid_time=slice(0, 6))

t1 = datetime.now()
comp = {"zlib": True, "complevel": 4}  # 1 being fastest, but lowest compression ratio, 9 being slowest but best compression ratio  
encoding = {
            var: {**comp, "chunksizes": (12, 1, 200, 200)} # 
            if len(test[var].dims) == 4 else {**comp, "chunksizes": (12, 200, 200)}
            for var in test.data_vars
        }
# encoding = {var: {**comp, "chunksizes": None} for var in daily_hrrr.data_vars}
test.to_netcdf(os.path.join(save_folder, "hrrr_test.nc"), encoding=encoding)

t2 = datetime.now()

print (t2-t1)

0:03:08.724218


In [7]:
import h5py

t1 = datetime.now()

hrrr = daily_hrrr

# Replace u and v at pressure levels - does not save disk space
hrrr['u_500hPa'] = hrrr['u'].sel(isobaricInhPa=500.0)  # Selecting u at 500 hPa
hrrr['v_500hPa'] = hrrr['v'].sel(isobaricInhPa=500.0)  # Selecting v at 500 hPa

hrrr['u_300hPa'] = hrrr['u'].sel(isobaricInhPa=300.0)  # Selecting u at 300 hPa
hrrr['v_300hPa'] = hrrr['v'].sel(isobaricInhPa=300.0)  # Selecting v at 300 hPa

hrrr['u_250hPa'] = hrrr['u'].sel(isobaricInhPa=250.0)  # Selecting u at 300 hPa
hrrr['v_250hPa'] = hrrr['v'].sel(isobaricInhPa=250.0)  # Selecting v at 300 hPa

hrrr['u_700hPa'] = hrrr['u'].sel(isobaricInhPa=700.0)  # Selecting u at 500 hPa
hrrr['v_700hPa'] = hrrr['v'].sel(isobaricInhPa=700.0)  # Selecting v at 500 hPa

hrrr['u_1000hPa'] = hrrr['u'].sel(isobaricInhPa=1000.0)  # Selecting u at 300 hPa
hrrr['v_1000hPa'] = hrrr['v'].sel(isobaricInhPa=1000.0)  # Selecting v at 300 hPa

hrrr['u_850hPa'] = hrrr['u'].sel(isobaricInhPa=850.0)  # Selecting u at 300 hPa
hrrr['v_850hPa'] = hrrr['v'].sel(isobaricInhPa=850.0)  # Selecting v at 300 hPa

hrrr['u_925hPa'] = hrrr['u'].sel(isobaricInhPa=925.0)  # Selecting u at 300 hPa
hrrr['v_925hPa'] = hrrr['v'].sel(isobaricInhPa=925.0)  # Selecting v at 300 hPa


# Step 3: Drop the original 'u' and 'v' variables
hrrr = hrrr.drop_vars(['u', 'v'])
hrrr = hrrr.drop_vars(['isobaricInhPa'])


#### Reduce netcdf to valid_time - coordinates (coordinates = time * (x*y)) - this does not save any storage space in netcdf either,
ds = hrrr

# Get original spatial shape
y_size, x_size = ds.sizes['y'], ds.sizes['x']
num_points = y_size * x_size  # Total number of spatial points

# Flatten latitude and longitude, then stack into a (2, x*y) shape
lat_flat = ds['latitude'].values.ravel()  # Flatten to (x*y,)
lon_flat = ds['longitude'].values.ravel()  # Flatten to (x*y,)
coordinates = np.vstack([lat_flat, lon_flat])  # Shape (2, x*y)

# Reshape all spatial variables to match new dimensions (valid_time, num_points)
new_data_vars = {}
for var_name, var_data in ds.data_vars.items():
    if 'y' in var_data.dims and 'x' in var_data.dims:
        # Flatten spatial dimensions while keeping valid_time
        new_data_vars[var_name] = (('valid_time', 'coordinates'), var_data.values.reshape(len(ds['valid_time']), num_points))
    elif 'valid_time' in var_data.dims:
        # Keep variables that do not depend on (y, x) unchanged
        new_data_vars[var_name] = var_data

# Create a new xarray dataset with modified dimensions
new_ds = xr.Dataset(
    data_vars=new_data_vars,
    coords={
        'valid_time': ds['valid_time'],  # Keep valid_time
        'coordinates': (('dim', 'coordinates'), coordinates)  # New 2-row coordinate variable
    }
)


# Create a new HDF5 file to save the data
h5_file = os.path.join(save_folder, f"hrrr_test.h5") 

with h5py.File(h5_file, 'w') as h5f:

    # Save "coordinates" dataset (shape: 2, x*y) with compression
    h5f.create_dataset('coordinates', data=new_ds['coordinates'].values, 
                        compression='gzip', compression_opts=9)
    
    # Save "valid_time" (convert datetime to string format for HDF5 compatibility)
    valid_time_str = new_ds['valid_time'].astype(str).values  # Convert to string
    h5f.create_dataset('valid_time', data=valid_time_str.astype('S'), 
                        compression='gzip', compression_opts=9)  # Store as bytes
    
    # Save all transformed data variables (valid_time, coordinates) with compression
    for var_name, var_data in new_ds.data_vars.items():
        h5f.create_dataset(var_name, data=var_data.values, 
                            compression='gzip', compression_opts=9)


t2 = datetime.now()

print (t2-t1)

MissingDimensionsError: 'coordinates' has more than 1-dimension and the same name as one of its dimensions ('dim', 'coordinates'). xarray disallows such variables because they conflict with the coordinates used to label dimensions.

In [8]:
new_data_vars

{'t2m': (('valid_time', 'coordinates'),
  array([[     nan,      nan,      nan, ...,      nan,      nan,      nan],
         [     nan,      nan,      nan, ...,      nan,      nan,      nan],
         [     nan,      nan,      nan, ...,      nan,      nan,      nan],
         ...,
         [     nan,      nan,      nan, ...,      nan,      nan,      nan],
         [     nan,      nan,      nan, ...,      nan,      nan,      nan],
         [291.6185, 291.6185, 291.6185, ..., 274.431 , 274.3685, 274.3685]],
        dtype=float32)),
 'sp': (('valid_time', 'coordinates'),
  array([[    nan,     nan,     nan, ...,     nan,     nan,     nan],
         [    nan,     nan,     nan, ...,     nan,     nan,     nan],
         [    nan,     nan,     nan, ...,     nan,     nan,     nan],
         ...,
         [    nan,     nan,     nan, ...,     nan,     nan,     nan],
         [    nan,     nan,     nan, ...,     nan,     nan,     nan],
         [101760., 101760., 101760., ..., 101140., 101140., 1